- plot for tissuetype acc at X 
- cos+jacc and 3methods ---- genes on x axis
- subsample a few times for accuracy?

In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = True

In [ ]:
import matplotlib as mpl
# mpl.rc("figure", dpi=100)

In [ ]:
# Data preparation
import cellsaw
import cellsaw.io_utils
#import cellsaw.preprocess
import notebookhelper

# loading data-> [[anndata]] 
filenames = notebookhelper.filenames
path = '/home/ubuntu/data/scdata/'
datasets = [cellsaw.io_utils.read(sample_size=1000, 
                            sampleseed = seed,
                            suffix = '.h5',
                            dir = path, 
                            datasets = filenames) 
                                for seed in [42,1337,31337,501,404]]

# get genescores according to different pp methods 
# note how seurat is first so we can calculate values 
# before normalizing as normalize is 'auto'
preprocessingmethods = ['seurat_v3','meanexpression','cell_ranger','seurat', 'natto']
_ = [[cellsaw.preprocess.annotate_genescores(d, selector = pp) for d in datasets] for pp in preprocessingmethods]


In [ ]:
# lets collect data in a gigantic dataframe
# how did i do this before? 
import cellsaw.similarity.measures

def mklist(rep_id,datasets,preprocessing,filenames):
    res = [ (rep_id,'cosine',genecount,preprocessing,
             *cellsaw.similarity.measures.adata_to_score(datasets,genecount, preprocessing, 'cosine', filenames)) 
                  for genecount in range(400,1000,50) ]
    res += [ (rep_id,'jaccard',genecount,preprocessing,
              *cellsaw.similarity.measures.adata_to_score(datasets,genecount, preprocessing, 'jaccard', filenames)) 
                  for genecount in range(1000,3000,200) ]
    return res

result = [line for i,d in enumerate(datasets) for pp in preprocessingmethods for line in mklist(i,d,pp, filenames)]

In [ ]:
# do plots oO 
